<a href="https://colab.research.google.com/github/Viktory4121/MAPI/blob/main/solving_SLAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Решение СЛАУ (псевдообращение методом Фаддеева)

In [6]:
import numpy as np

#A = np.array([[-9,3,4,1],[3,0,1,-10]])
#A = np.array([[8,-5,1,4,-9],[2,5,-1,4,2],[-1,-2,-6,9,5],[6,2,0,-7,2],[6,2,5,9,1],[-5,-1,4,8,-2]])
#A = np.array([[10,-62,11,6],[0,24,-5,-7],[1,8,4,12]])
#A = np.array([[-1,2,5],[5,2,7],[0,-7,-1],[6,5,4],[1,6,2],[1,2,3],[-7,-4,-1]])
#A = np.array([[1,2,7,-2,1],[6,0,4,-2,5],[-4,-8,3,2,1],[7,0,-4,3,2]])
A = np.array([[12,7,9,36],[98,3,5,2],[6,78,3,1],[1,3,10,41]])
B = np.array([[2],[-7],[4],[1]])

def print_matrix(A):
  for row in A:
    for i in row:
      #print("{} & ".format(str(round(i))), end = "" )
      print("{}\t\t".format(str(round(i))), end = "" )
    print()

def print_float_matrix(A):
  for row in A:
    for i in row:
      #print("{:6f} & ".format(i), end = "" )
      print("{:6f}\t\t".format(i), end = "" )
    print()

#--------------------------------
n = len(A[0])
A_T = A.transpose()
A_T_A = A_T.dot(A) #перемножены транспонированная матрица и обычная
F = np.diag(np.diag(np.ones([n,n]))) #единичная матрица
mu = sum(np.diag(A_T_A)) #это фи1 (сумма значений матрицы A_T_A по диагонали)
#--------------------------------

def Faddeev(A_T_A, F, mu, A_T):
    n = len(F)
    I = np.diag(np.diag(np.ones([n,n])))
    i = 2
    while mu != 0:
        if sum(np.diag(A_T_A.dot(mu*I - A_T_A.dot(F))))/float(i) == 0:
            _mu = mu
            _F = F
        F = mu*I - A_T_A.dot(F)
        mu = sum(np.diag(A_T_A.dot(F)))/float(i)
        
        i+=1

    print("РЕЗУЛЬТАТ:\nA*: 1/", _mu)
    A_psevv = _F.dot(A_T)
    print_matrix(A_psevv)

    print("\nили A*:")
    print_float_matrix(A_psevv/_mu)
    return(A_psevv/_mu, _mu)

A_psev, muu = Faddeev(A_T_A, F, mu, A_T)

РЕЗУЛЬТАТ:
A*: 1/ 17346470436.0
1856264364		-27394848		-102994092		-1626042276		
1312186878		-158442318		154886256		-1148212908		
-41125857030		4539247290		2137851792		35836939188		
9889408422		-1094871978		-530248356		-8193957084		

или A*:
0.107011		-0.001579		-0.005937		-0.093739		
0.075646		-0.009134		0.008929		-0.066193		
-2.370849		0.261681		0.123244		2.065950		
0.570111		-0.063118		-0.030568		-0.472370		


In [18]:
def check_solvability(A, A_psev, B):
  n = len(A)
  I = np.diag(np.diag(np.ones([n,n])))
  print("Проверка на совместность:")
  print("A A^+ =")
  print_float_matrix(A.dot(A_psev))
  print("A A^+ b = ")
  print_float_matrix((A.dot(A_psev)).dot(B))
  print("B =")
  print_float_matrix(B)

  private = I - A_psev.dot(A)
  return private

def slae(A, A_psev, B):
  prive = check_solvability(A, A_psev, B)
  X = A_psev.dot(B) #общее решение

  print("\n\nРЕШЕНИЕ СЛАУ:\nX_общее =")
  print_float_matrix(X)
  print()
  print("X_частное = ")
  print_float_matrix(prive)

  return (X, prive) #общее и частное

x_o, x_pr = slae(A, A_psev, B)

Проверка на совместность:
A A^+ =
1.000000		0.000000		0.000000		0.000000		
-0.000000		1.000000		0.000000		0.000000		
-0.000000		0.000000		1.000000		-0.000000		
0.000000		-0.000000		0.000000		1.000000		
A A^+ b = 
2.000000		
-7.000000		
4.000000		
1.000000		
B =
2.000000		
-7.000000		
4.000000		
1.000000		


РЕШЕНИЕ СЛАУ:
X_общее =
0.107588		
0.184752		
-4.014540		
0.987404		

X_частное = 
0.000000		0.000000		0.000000		0.000000		
-0.000000		-0.000000		0.000000		0.000000		
0.000000		0.000000		0.000000		0.000000		
0.000000		-0.000000		-0.000000		-0.000000		
